### Imports

In [32]:
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import warnings

# Input Raw File and Decoder File

In [2]:
file = "LIV_53_QCHW2_DNS-LIVTKCOD_LCRVCOD250-DNS_raw20250130133124.csv"
decoder_file = "QC WAFER_LAYOUT 24Dec.csv"
# Define the current script's directory
current_dir = Path(os.getcwd())

# Move to the root directory
root_dir = current_dir.parents[0]  # Adjust the number based on your folder structure

# Define the path to the required subfolder
file_path = root_dir / "LIV_Raw_Files" / file
decoder_file_path = root_dir / "decoders" / decoder_file

# Read the CSV file
df = pd.read_csv(file_path, skiprows=19)

# Get the directory of the file
folder = file_path.parent
print(f"Directory: {folder}")

# df.head(5)

Directory: c:\Users\762093\OneDrive - Seagate Technology\Documents\LIV\liv-cod-algorithm\LIV_Raw_Files


# Transform Data to Desired Raw Sweep Format

- selects required columns
- transposes
- stacks data in tall format
- adds in device coords from decoder file

In [34]:
# Transforming Raw File Code


warnings.filterwarnings("ignore")

# Get column names
col_names = df.columns



# Find columns containing "Vf" or "PD"



selected_cols = [col for col in col_names if "Vf" in col or "PD" in col]



# Subset the data frame with selected columns



df_subset = df[selected_cols]



# Find and delete columns containing "Vf@" or "PD@"



cols_to_delete = [col for col in df_subset.columns if "Vf@" in col or "PD@" in col]



df_subset.drop(columns=cols_to_delete, inplace=True)



# Transpose the data frame
df_transposed = df_subset.transpose()

# Reset index to make the transposed columns into rows
df_transposed.reset_index(inplace=True)

# Add a new row at the top with the desired titles
new_columns = ["Label"] + list(range(1, len(df_transposed.columns)))
df_transposed.columns = new_columns
df_transposed.loc[-1] = new_columns  # Add the new row at the top
df_transposed.index = df_transposed.index + 1  # Shift the index
df_transposed = df_transposed.sort_index()  # Sort by index to place the new row at the top

# Display the first 10 rows of df_transposed
df_transposed.head(10)
# df_subset.head(10)

# Split transposed table into Vf and PD data tables
df_vf = df_transposed[df_transposed["Label"].str.contains("Vf")]
df_pd = df_transposed[df_transposed["Label"].str.contains("PD")]
# df_vf.head(10)
# df_pd.head(10)

# Drop the 'Label' column
df_vf.drop(columns=["Label"], inplace=True)
df_pd.drop(columns=["Label"], inplace=True)

# learn data dimensions
n_meas = df_vf.shape[0]
print(f"Number of Current Measurements per Device: {n_meas}")
n_devices = df_vf.shape[1]
print(f"Number of Devices: {n_devices}")

# Concatenate all Voltage columns into one
df_concat_vf = pd.concat([df_vf[col] for col in df_vf.columns], ignore_index=True).to_frame(
    name="Vf"
)

# Create TOUCHDOWN column
df_concat_vf["TOUCHDOWN"] = [i // n_meas + 1 for i in range(n_meas * n_devices)]

# Concatenate all PD columns into one
df_concat_pd = pd.concat([df_pd[col] for col in df_pd.columns], ignore_index=True).to_frame(
    name="PD"
)

# Cartesian join of Vf and PD data tables
df_raw_sweeps = pd.concat([df_concat_vf, df_concat_pd], axis=1)
# df_raw_sweeps.head(270)

# Add device coordinates from original RAW file


if "TOUCHDOWN" in df.columns and "STX_WAFER_X_UM" in df.columns and "STX_WAFER_Y_UM" in df.columns:
    df_raw_sweeps = df_raw_sweeps.merge(
        df[["TOUCHDOWN", "STX_WAFER_X_UM", "STX_WAFER_Y_UM"]], on="TOUCHDOWN", how="left"
    )
else:
    print("Required columns for merging device coordinates are missing in the original RAW file.")


if decoder_file_path.exists():
    df_decoder = pd.read_csv(decoder_file_path)
    # Update with decoder to get TE_LABEL etc.
    if "YMIN" in df_decoder.columns and "XMIN" in df_decoder.columns:

        df_raw_sweeps = df_raw_sweeps.merge(
            df_decoder[["YMIN", "XMIN", "TE_LABEL", "TYPE"]],
            left_on=["STX_WAFER_Y_UM", "STX_WAFER_X_UM"],
            right_on=["YMIN", "XMIN"],
            how="left",
        ).drop(columns=["YMIN", "XMIN"])
    else:
        print("Required columns for merging decoder data are missing in the decoder file.")
else:
    print(f"Decoder file not found at {decoder_file_path}")


# Rename the columns
df_raw_sweeps.rename(columns={"STX_WAFER_X_UM": "X_UM", "STX_WAFER_Y_UM": "Y_UM"}, inplace=True)


# Add current column as a repeating sequence of length n_meas
df_raw_sweeps["LDI_mA"] = [i % n_meas + 1 for i in range(len(df_raw_sweeps))]


# Display the first 10 rows of df_raw_sweeps
df_raw_sweeps.head(379500)

Number of Current Measurements per Device: 250
Number of Devices: 1518


,Vf,TOUCHDOWN,PD,X_UM,Y_UM,TE_LABEL,TYPE,LDI_mA
0,1.276144,1,0.171087,-32339,-48416,64N6V,BL LASER,1
1,1.398021,1,0.168066,-32339,-48416,64N6V,BL LASER,2
2,1.446096,1,0.168066,-32339,-48416,64N6V,BL LASER,3
3,1.475857,1,0.171087,-32339,-48416,64N6V,BL LASER,4
4,1.494875,1,0.169577,-32339,-48416,64N6V,BL LASER,5
...,...,...,...,...,...,...,...,...
379495,4.772126,1518,0.166555,42094,-60632,70B06,NO BFMM TK,246
379496,4.770936,1518,0.163534,42094,-60632,70B06,NO BFMM TK,247
379497,4.769746,1518,0.160512,42094,-60632,70B06,NO BFMM TK,248
379498,4.768556,1518,0.159001,42094,-60632,70B06,NO BFMM TK,249
